In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F


torch.manual_seed(1337)

# A demonstration that with a causal mask, the an element of the output is not affected by future input elements

In [25]:
from torch.autograd import Variable 

# lets try with einsum
B, T, C = 4, 8, 32 # batch, time, channels

g = torch.Generator().manual_seed(2147483647) # for reproducibility

In [62]:
# x = torch.randn(B, T, C) # (B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)   # (C, 16)
query = nn.Linear(C, head_size, bias=False) # (C, 16)
value = nn.Linear(C, head_size, bias=False) # (C, 16)

def context(x):
    # The input is a batch of sequences of vectors of size C
    # A vector of size C: (C)
    # A sequence of vectors of size C: (T, C). Think of a stack of vectors

    k = key(x)   # (B, T, C) @ (C, 16) = (B, T, 16)
    q = query(x) # (B, T, C) @ (C, 16) = (B, T, 16)
    attn = torch.einsum("b i h , b j h -> b i j", q, k) * C ** -0.5

    tril = torch.tril(torch.ones(T, T))
    print(tril.shape)
    attn = attn.masked_fill(tril == 0, float('-inf'))


    attn = F.softmax(attn, dim=-1) 



    v = value(x)
    # The output is a batch of sequences of vectors of size H (16)
    return torch.einsum("b t t, b t h -> b t h", attn, v)

x = torch.randn(B, T, C, requires_grad=True)

context(x)[0][-2].sum().backward()
print(x.grad[0][-1].sum()) # == 0
print(x.grad[0][-2].sum()) # != 0
print(x.grad[0][-3].sum()) # != 0
print(x.grad[0][-4].sum()) # != 0

tensor(0.)
tensor(0.4878)
tensor(-0.0004)
tensor(-0.0005)


In [ ]:
out

NameError: name 'out' is not defined